# Solution 5 - Principal Component Analysis
## Data Analysis
### FINM August Review 

Mark Hendricks

hendricks@uchicago.edu

## 1 Principal Components

Use the single-name equity data from the data set mentioned above. (That is, leave out SPY and SHV.)

1. Calculate the principal components of the return series.


2. Report the eigenvalues associated with these principal components. Report each eigenvalue as a percentage of the sum of all the eigenvalues. This is the total variation each PCA explains.


3. How many PCs are needed to explain 75% of the variation?


4. Calculate the correlation between the first (largest eigenvalue) principal component with each of the 22 single-name equities. Which correlation is highest?


5. Calculate the correlation between the SPY and the first, second, third principal components.

# <span style='color:red'>Solution 1</span>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")

In [2]:
FILEIN = '../data/spx_returns_weekly.xlsx'
rets = pd.read_excel(FILEIN, sheet_name=2, index_col=0)
spy = pd.read_excel(FILEIN, sheet_name=3, index_col=0)[['SPY']]

NSECS = rets.shape[1]

In [3]:
# data = pd.read_excel("single_name_return_data.xlsx", sheet_name="total returns").set_index("Date")
# equities = data.drop(columns=['SPY', 'SHV'])
# spy = data[['SPY']]
# rf = data[['SHV']]

##### 1.

In [4]:
# Steps to calculate PC of the return series:

pca = PCA(svd_solver='full')
pca.fit(rets.values)

pca_factors = pd.DataFrame(pca.transform(rets.values), 
                           columns=['PC {}'.format(i+1) for i in range(pca.n_components_)], 
                           index = pd.to_datetime(rets.index))
display(pca_factors.tail().style.format('{:,.4f}'))

##### 2.

In [5]:
explained_var = pd.DataFrame(data = pca.explained_variance_ratio_,
                                 index = pca_factors.columns, 
                                 columns = ['Explained Variance'])
explained_var['Cumulative Explained Variance'] = explained_var['Explained Variance'].cumsum()

display(explained_var.style.format('{:,.2%}'))

##### 3.

Looking at the cumulative percentage of explained variance, the first 5 Principal Component Factors are enough to explain over 55% of the variation in the single name equities.

Just the first PC explains over 40%!

###### 4.

In [6]:
rets_corr = rets.copy()
rets_corr.insert(0, 'PC 1', pca_factors['PC 1'])

corr_table = pd.DataFrame(rets_corr.corr().iloc[0, 1:]).sort_values('PC 1').style.format('{:,.2%}')
display(corr_table)

##### 5. 

In [7]:
factors = pca_factors[['PC 1', 'PC 2', 'PC 3']]
factors['SPY'] = spy
print('Correlation with SPY:')
display(factors.corr().iloc[3, :3])

Correlation with SPY:


PC 1   -0.941980
PC 2   -0.232393
PC 3   -0.096286
Name: SPY, dtype: float64

***

## 2 PCR and PLS

1. Principal Component Regression (PCR) refers to using PCA for dimension reduction, and then utilizing the principal components in a regression. Try this by regressing SPY on the first 3 PCs calculated in the previous section. Report the r-squared.


2. Calculate the Partial Least Squares estimation of SPY on the 22 single-name equities. Model it for 3 factors. Report the r-squared.


3. Compare the results between these two approaches and against penalized regression seen in the past homework.

# <span style='color:red'>Solution 2</span>

###### 1.

In [8]:
X_PCR = factors[['PC 1', 'PC 2', 'PC 3']]

model_PCR = LinearRegression().fit(X_PCR, spy)
print('PCR R-squared: ' + str(round(model_PCR.score(X_PCR, spy),3)))

PCR R-squared: 0.951


###### 2.

In [9]:
X_PLS = rets
y_PLS = spy

model_PLS = PLSRegression(n_components=3).fit(X_PLS, y_PLS)

print('PLS R-squared: ' + str(round(model_PLS.score(X_PLS, y_PLS),3)))

PLS R-squared: 0.982


##### 3. 

PCR and PLS both seek to maximize the ability to explain the variation in y variable, and therefore they will have high $R^2$ in-sample. When using LASSO or Ridge as our model, we are conservatively forming factors, and penalizing for additional factors. This makes in-sample $R^2$ lower as we saw in Homework #5, but may make more robust OOS predictions.

***

### <span style='color:red'>Footnote</span>

For those intrested in the implementation of PCA, here is a simple code in Python:

In [10]:
# Steps to calculate PC of the return series:

### 1. Center the return series (mean = 0)

rets_centered = rets - np.mean(rets, axis = 0)

### 2. Calculate the Covariance Matrix

cov = np.cov(rets_centered , rowvar = False)

### 3. Calculating Eigenvalues and Eigenvectors of the covariance matrix

eigen_values , eigen_vectors = np.linalg.eigh(cov)

### 4. Sort the eigenvalues and corresponding eigenvectors in descending order

sorted_index = np.argsort(eigen_values)[::-1]
sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:,sorted_index]

### 5. Transform the Data

PCA_factors = np.dot(sorted_eigenvectors.transpose(),rets_centered.transpose()).transpose()
display(pd.DataFrame(PCA_factors))

0         1         2         3         4         5         6    \
0    0.713621 -0.074633 -0.270530  0.134429  0.150918  0.197287 -0.012905   
1   -0.217894 -0.072781  0.010706 -0.185791 -0.164729  0.063318  0.075381   
2   -0.342672  0.060118 -0.061065 -0.232358  0.047794  0.099135  0.297087   
3    0.660801  0.156921 -0.248732  0.021150  0.217021 -0.038635  0.404337   
4    0.424689 -0.106462  0.016677  0.090441  0.212052  0.066341 -0.120474   
..        ...       ...       ...       ...       ...       ...       ...   
387 -0.447157 -0.211018  0.090062  0.034896  0.051949  0.063731  0.067914   
388  0.547718 -0.017491 -0.020469  0.057174  0.042458  0.026121 -0.114772   
389 -0.703391  0.025490  0.043145  0.058523  0.009740  0.003219  0.136065   
390  0.181140  0.252394 -0.076085  0.048549 -0.025807 -0.029382 -0.005921   
391 -0.415835 -0.217840  0.097220  0.015662 -0.074206  0.048727  0.091546   

          7         8         9    ...           440           441  \
0    0.007153 -0.069679 -0.136514  ... -2.116363e-16  4.093947e-16   
1   -0.065257  0.043856 -0.046646  ... -1.249001e-16 -7.389922e-16   
2    0.014235 -0.117428 -0.057120  ...  8.638923e-16 -6.036838e-16   
3    0.111642 -0.005754  0.053273  ... -6.375109e-16  6.175616e-16   
4   -0.055499 -0.321965 -0.124382  ...  1.297573e-15  2.116363e-16   
..        ...       ...       ...  ...           ...           ...   
387 -0.062480 -0.131404 -0.048774  ... -4.492934e-16  4.284767e-16   
388  0.136683 -0.069883 -0.121480  ...  2.120699e-15 -5.244286e-16   
389 -0.047367  0.070771 -0.024069  ... -1.597680e-15  1.717376e-16   
390 -0.091143  0.046754  0.043876  ...  4.208439e-15 -6.297046e-16   
391  0.033381  0.062910 -0.129487  ...  5.572799e-16 -4.527628e-16   

              442           443           444           445           446  \
0   -4.831205e-16  6.739401e-16  1.564721e-15 -1.665335e-16 -4.406198e-16   
1   -2.940356e-16  3.612562e-16 -1.010476e-15  3.456437e-16  2.844947e-16   
2   -1.526557e-16  3.226586e-16  3.712308e-16 -1.528291e-15 -3.191891e-16   
3    3.556183e-17  6.245005e-17  1.658829e-16  3.044440e-16  2.289835e-16   
4   -2.359224e-16 -4.041906e-16  2.979388e-16  3.903128e-18 -6.539908e-16   
..            ...           ...           ...           ...           ...   
387 -5.898060e-17  7.285839e-17 -7.285839e-17  6.800116e-16  8.673617e-18   
388  1.268950e-15  1.363493e-15  1.096345e-15  2.775558e-16 -1.051242e-15   
389 -2.320193e-16  1.075529e-16 -4.510281e-17  1.280226e-15 -1.763346e-15   
390  8.456777e-16 -6.696033e-16  3.295975e-16  6.418477e-17 -5.915407e-16   
391  9.298118e-16  4.087442e-16 -3.191891e-16  5.544610e-16 -1.050375e-15   

              447           448           449  
0    3.469447e-17  4.492934e-16 -9.228729e-16  
1    1.318390e-15 -1.181347e-15  2.081668e-17  
2   -2.966377e-16  9.194034e-17  7.129713e-16  
3   -2.220446e-16  1.665335e-16 -6.774095e-16  
4   -2.220446e-16  2.393918e-16  5.846018e-16  
..            ...           ...           ...  
387 -4.531965e-16  1.017415e-15 -1.618497e-15  
388  3.937822e-16  1.179612e-16  7.511353e-16  
389  6.383782e-16 -1.115427e-15 -1.144917e-16  
390  6.279699e-16  1.557782e-15 -1.595946e-16  
391  4.651227e-16 -8.552187e-16 -1.336171e-15  

[392 rows x 450 columns]